Load Data

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd
import tensorflow as tf
import numpy as np

#df = pd.read_csv("/content/drive/MyDrive/"+"data_df" + ".csv", skiprows=[0])
df = pd.read_csv("/content/"+"data_df" + ".csv", skiprows=[0])
dataset = df.values
X = dataset[:,1:61].astype(float)
Y = dataset[:,61]
X = (X+90)/180
# randomize
num_inputs = len(X)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)
X = X[randomize]
Y = Y[randomize]

split = int(0.9* num_inputs)
X_train, X_test = np.split(X, [split])
Y_train, Y_test = np.split(Y, [split])

MLP

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100, activation='relu', input_dim = 60)) # relu is used for performance
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid')) # we either have fall or not
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=100)

Epoch 1/100
15/15 [==============================] - 0s 3ms/step - loss: 0.6610 - accuracy: 0.6978
Epoch 2/100
15/15 [==============================] - 0s 3ms/step - loss: 0.6503 - accuracy: 0.7478
Epoch 3/100
15/15 [==============================] - 0s 2ms/step - loss: 0.6329 - accuracy: 0.7478
Epoch 4/100
15/15 [==============================] - 0s 2ms/step - loss: 0.6099 - accuracy: 0.7478
Epoch 5/100
15/15 [==============================] - 0s 2ms/step - loss: 0.5996 - accuracy: 0.7478
Epoch 6/100
15/15 [==============================] - 0s 2ms/step - loss: 0.5857 - accuracy: 0.7478
Epoch 7/100
15/15 [==============================] - 0s 2ms/step - loss: 0.5715 - accuracy: 0.7478
Epoch 8/100
15/15 [==============================] - 0s 2ms/step - loss: 0.5498 - accuracy: 0.7478
Epoch 9/100
15/15 [==============================] - 0s 2ms/step - loss: 0.5230 - accuracy: 0.7565
Epoch 10/100
15/15 [==============================] - 0s 2ms/step - loss: 0.4932 - accuracy: 0.7848
Epoch 11/

Test

In [ ]:
predictions = model.predict(X_test)
predictions = predictions >= 0.5
from sklearn.metrics import classification_report
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.92      0.92      0.92        36
         1.0       0.81      0.81      0.81        16

    accuracy                           0.88        52
   macro avg       0.86      0.86      0.86        52
weighted avg       0.88      0.88      0.88        52



Save model as Arduino header

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("model.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("model.tflite")
print("Model is %d bytes" % basic_model_size)

!apt-get install xxd
!echo "const unsigned char model[] = {" > /content/model.h
!cat model.tflite | xxd -i      >> /content/model.h
!echo "};"

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

INFO:tensorflow:Assets written to: /tmp/tmpg7072l1j/assets


INFO:tensorflow:Assets written to: /tmp/tmpg7072l1j/assets


Model is 67040 bytes
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  xxd
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 49.9 kB of archives.
After this operation, 201 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 xxd amd64 2:8.0.1453-1ubuntu1.8 [49.9 kB]
Fetched 49.9 kB in 0s (181 kB/s)
Selecting previously unselected package xxd.
(Reading database ... 155629 files and directories currently installed.)
Preparing to unpack .../xxd_2%3a8.0.1453-1ubuntu1.8_amd64.deb ...
Unpacking xxd (2:8.0.1453-1ubuntu1.8) ...
Setting up xxd (2:8.0.1453-1ubuntu1.8) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
};
Header file, model.h, is 413,445 bytes.

Open the side panel (ref